In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

In [2]:
import pandas as pd
import dask.dataframe as dd

X = pd.DataFrame(zip(range(100), range(100)), columns=['x','y'])
X_dask = dd.from_pandas(X, npartitions=1)
X_spark = spark.createDataFrame(X)

In [3]:
from dfender import distributable

class Test:
    @distributable(pd.Series)
    def predict(self, X: pd.Series) -> pd.Series:
        return X == 2
        # return (pd.to_datetime('2000-01-01').to_period('M') + X).dt.to_timestamp()

    @distributable((pd.Series, pd.DataFrame))
    def rowmax(self, X: pd.DataFrame):
        return X.max(axis=1)


est = Test()
# est.predict(X['x'])
# est.predict(X_dask['x']).compute()
# est.predict(X_spark.select(['x']))
# est.rowmax(X)
# est.rowmax(X_)
# Test.test.funcs
est.predict.funcs

{(pandas.core.series.Series,): <function __main__.Test.predict(self, X: pandas.core.series.Series) -> pandas.core.series.Series>,
 (dask.dataframe.core.Series,): <function dfender.materializers.core.MaterializerInterface.map.<locals>.wrapper(_, df: 'Any', *args, **kwargs)>,
 (pyspark.sql.dataframe.DataFrame,): <function dfender.materializers.core.MaterializerInterface.map.<locals>.wrapper(_, df: 'Any', *args, **kwargs)>}

In [4]:
est.predict(X['x'])

0     False
1     False
2      True
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: x, Length: 100, dtype: bool

In [7]:
est.predict(X_dask['x']).compute()

0     False
1     False
2      True
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: x, Length: 100, dtype: bool

In [8]:
est.predict(X_spark['x'])

NameError: name 'X_spark' is not defined